<a href="https://colab.research.google.com/github/jimmy-pink/colab-playground/blob/main/pre-trained/EfficientNetB3-FolderIconRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用EfficientNetB3微调以解决FolderIcon二分类问题



### 问题分析
#### 背景
已在VGG16预训练模型上微调，在对2170多个folder icon预测，存在以下问题：
- 发现准确率仅为88%-91%
- 更要命的是基于VGG16训练出来的模型经常犯超低级错误，将毫不相干的图像判断为真。   

鉴于VGG16是非常老的模型，现以EfficientNetB3再训练一版，观察模型性能有无明显提升。



In [ ]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam

### 数据准备

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 挂载 Google Drive
base_dir = '/content/drive/MyDrive/Google-AI-Studio/data/folder-icon-images/'  # Google Drive 中的图像文件夹路径
train_dir=base_dir+'is_folder'
drive_train_validate_dir = base_dir + "train_validate"

In [ ]:
# 设置数据增强
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)

# 使用 flow_from_directory 加载训练数据
# 数据增强 + 归一化
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.3  # 30% 作为验证集
)

# 训练集生成器
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='binary',  # 二分类用 binary，多分类用 categorical
    subset='training'  # 指定是训练集
)

# 验证集生成器
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # 指定是验证集
)
print(f"找到的训练样本数: {train_generator.samples}")
print(f"找到的验证样本数: {validation_generator.samples}")
images, labels = next(train_generator)
print("图像形状:", images.shape)
print("标签形状:", labels.shape)

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
# 获取训练集的真实标签（需确保 train_generator.classes 是整数标签 0/1）
y_train = train_generator.classes
# 计算类别权重（classes 需是 NumPy 数组）
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),  # 自动提取唯一类别（如 [0, 1]）
    y=y_train
)
# 转换为字典格式
class_weights = {i: weight for i, weight in enumerate(class_weights)}
print("类别权重:", class_weights)

### 建模与训练

In [ ]:
base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=(300, 300, 3))

# 冻结前 90% 的层
freeze_until = int(len(base_model.layers) * 0.9)
for i, layer in enumerate(base_model.layers):
    layer.trainable = i >= freeze_until

In [ ]:
from tensorflow.keras.optimizers import AdamW

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.4),  # 添加Dropout层，丢弃50%的神经元
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # 最后一层对于二分类任务使用sigmoid激活
])

optimizer = AdamW(learning_rate=1e-4, weight_decay=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') >= 0.98 and logs.get('val_loss') < 0.2 ):
            self.model.stop_training = True
callbacks = myCallback()
# 训练模型
history = model.fit(train_generator,
          epochs=30,
          validation_data=validation_generator,
          callbacks=[callbacks])